![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings_JSL/Healthcare/1.4.Resume_MedicalNer_Model_Training.ipynb)

# 1.4 Resume MedicalNer Model Training

Steps:
- Train a new model for a few epochs.
- Load the same model and train for more epochs on the same taxnonomy, and check stats.
- Train a model already trained on a different data

## Colab Setup

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs 

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import * 
# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
# Make sure to restart your notebook afterwards for changes to take effect
jsl.install()

In [ ]:
from johnsnowlabs import * 
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Automatically load license data and start a session with all jars user has access to
spark = jsl.start()

## Download Clinical Word Embeddings for training

In [2]:
clinical_embeddings = nlp.WordEmbeddingsModel.pretrained('embeddings_clinical', "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]


## Download Data for Training (NCBI Disease Dataset)

In [3]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NCBI_disease_official_test.conll
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NCBI_disease_official_train_dev.conll

In [4]:
# from sparknlp.training import CoNLL

training_data = nlp.CoNLL().readDataset(spark, 'NCBI_disease_official_train_dev.conll')

training_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Identification of...|[{document, 0, 89...|[{document, 0, 89...|[{token, 0, 13, I...|[{pos, 0, 13, NN,...|[{named_entity, 0...|
|The adenomatous p...|[{document, 0, 21...|[{document, 0, 21...|[{token, 0, 2, Th...|[{pos, 0, 2, NN, ...|[{named_entity, 0...|
|Complex formation...|[{document, 0, 63...|[{document, 0, 63...|[{token, 0, 6, Co...|[{pos, 0, 6, NN, ...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [5]:
test_data = nlp.CoNLL().readDataset(spark, 'NCBI_disease_official_test.conll')

test_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Clustering of mis...|[{document, 0, 10...|[{document, 0, 10...|[{token, 0, 9, Cl...|[{pos, 0, 9, NN, ...|[{named_entity, 0...|
|Ataxia - telangie...|[{document, 0, 13...|[{document, 0, 13...|[{token, 0, 5, At...|[{pos, 0, 5, NN, ...|[{named_entity, 0...|
|The risk of cance...|[{document, 0, 15...|[{document, 0, 15...|[{token, 0, 2, Th...|[{pos, 0, 2, NN, ...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



## Split the test data into two parts:
- We Keep the first part separate and use it for training the model further, as it will be totally unseen data from the same taxonomy.

- The second part will be used to testing and evaluating

In [8]:
(test_data_1, test_data_2) = test_data.randomSplit([0.5, 0.5], seed = 100)

# save the test data as parquet for easy testing
clinical_embeddings.transform(test_data_1).write.parquet('test_1.parquet')

clinical_embeddings.transform(test_data_2).write.parquet('test_2.parquet')

## Train a new model, pause, and resume training on the same dataset.

### Create a graph

We will use `TFGraphBuilder` annotator which can be used to create graphs in the model training pipeline. `TFGraphBuilder` inspects the data and creates the proper graph if a suitable version of TensorFlow (<= 2.7 ) is available. The graph is stored in the defined folder and loaded by the approach.

In [9]:
!pip install -q tensorflow==2.7.0
!pip install -q tensorflow-addons

In [10]:
graph_folder_path = "medical_ner_graphs"

ner_graph_builder = medical.TFGraphBuilder()\
    .setModelName("ner_dl")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setLabelColumn("label")\
    .setGraphFolder(graph_folder_path)\
    .setGraphFile("auto")\
    .setHiddenUnitsNumber(24)\
    .setIsMedical(True) # False -> if you want to use TFGraphBuilder with NerDLApproach

In [11]:
# You can create custom TensorFlow graph file (`.pb` extension) for NER model training externally by using the script below. 
# If this method is used, graph folder should be added to NerApproach as  `.setGraphFolder(graph_folder_path)` .

'''
medical.tf_graph.print_model_params("ner_dl")

graph_folder_path = "medical_ner_graphs"

medical.tf_graph.build("ner_dl",
               build_params={"embeddings_dim": 200,
                             "nchars": 128,
                             "ntags": 12,
                             "is_medical": 1},
                model_location=graph_folder_path,
                model_filename="auto")
'''

'\nmedical.tf_graph.print_model_params("ner_dl")\n\ngraph_folder_path = "medical_ner_graphs"\n\nmedical.tf_graph.build("ner_dl",\n               build_params={"embeddings_dim": 200,\n                             "nchars": 128,\n                             "ntags": 12,\n                             "is_medical": 1},\n                model_location=graph_folder_path,\n                model_filename="auto")\n'

### Train for 2 epochs

In [12]:
nerTagger = medical.NerApproach()\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setLabelColumn("label")\
    .setOutputCol("ner")\
    .setMaxEpochs(2)\
    .setLr(0.003)\
    .setBatchSize(8)\
    .setRandomSeed(0)\
    .setVerbose(1)\
    .setEvaluationLogExtended(True) \
    .setEnableOutputLogs(True)\
    .setIncludeConfidence(True)\
    .setTestDataset('./test_2.parquet')\
    .setGraphFolder(graph_folder_path)\
    .setOutputLogsPath('./ner_logs')

ner_pipeline = Pipeline(stages=[
    clinical_embeddings,
    ner_graph_builder,
    nerTagger
 ])

In [13]:
%%time
ner_model = ner_pipeline.fit(training_data)

TF Graph Builder configuration:
Model name: ner_dl
Graph folder: medical_ner_graphs
Graph file name: auto
Build params: {'ntags': 3, 'embeddings_dim': 200, 'nchars': 85, 'is_medical': True, 'lstm_size': 24}


Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


ner_dl graph exported to medical_ner_graphs/blstm_3_200_24_85.pb
CPU times: user 14.1 s, sys: 1.09 s, total: 15.2 s
Wall time: 2min 24s


In [14]:
!ls ner_logs/MedicalNerApproach*

ner_logs/MedicalNerApproach_0542631d735e.log
ner_logs/MedicalNerApproach_d84ef750529e.log


In [15]:
# Training Logs
! cat ner_logs/MedicalNerApproach*

Name of the selected graph: /content/medical_ner_graphs/blstm_3_200_24_85.pb
Training started - total epochs: 2 - lr: 0.003 - batch size: 8 - labels: 3 - chars: 84 - training examples: 6347


Epoch 1/2 started, lr: 0.003, dataset size: 6347


Epoch 1/2 - 49.27s - loss: 2027.7551 - avg training loss: 2.5506353 - batches: 795
Quality on test dataset: 
time to finish evaluation: 1.95s
Total test loss: 89.1850	Avg test loss: 1.3513
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 399	 27	 184	 0.9366197	 0.6843911	 0.79088205
B-Disease	 407	 98	 111	 0.80594057	 0.78571427	 0.79569894
tp: 806 fp: 125 fn: 295 labels: 2
Macro-average	 prec: 0.87128013, rec: 0.7350527, f1: 0.79739
Micro-average	 prec: 0.86573577, rec: 0.73206174, f1: 0.79330707


Epoch 2/2 started, lr: 0.0029850747, dataset size: 6347


Epoch 2/2 - 45.26s - loss: 974.6585 - avg training loss: 1.2259855 - batches: 795
Quality on test dataset: 
time to finish evaluation: 1.18s
Total test loss: 63.7578	Avg test loss: 0.9660
label	 tp

In [16]:
# Logs of 4 consecutive epochs to compare with 2+2 epochs on separate datasets from same taxonomy

#!cat ner_logs/MedicalNerApproach_4d3d69967c3f.log

### Evaluate

In [18]:
from sparknlp_jsl.eval import NerDLMetrics

pred_df = ner_model.stages[2].transform(clinical_embeddings.transform(test_data_2))

evaler = NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
           .withColumn("recall", F.round(eval_result["recall"],4))\
           .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+-----+----+-----+---------+------+------+
| entity|   tp|   fp|  fn|total|precision|recall|    f1|
+-------+-----+-----+----+-----+---------+------+------+
|Disease|418.0|103.0|97.0|515.0|   0.8023|0.8117|0.8069|
+-------+-----+-----+----+-----+---------+------+------+

+-----------------+
|            macro|
+-----------------+
|0.806949806949807|
+-----------------+

None
+-----------------+
|            micro|
+-----------------+
|0.806949806949807|
+-----------------+

None


### Save the model to disk

In [19]:
ner_model.stages[2].write().overwrite().save('models/NCBI_NER_2_epoch')

### Train using the saved model on unseen dataset
#### We use unseen data from same taxonomy

In [20]:
nerTagger = medical.NerApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(2)\
      .setLr(0.003)\
      .setBatchSize(8)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setTestDataset('/content/test_2.parquet')\
      .setOutputLogsPath('ner_logs')\
      .setGraphFolder(graph_folder_path)\
      .setPretrainedModelPath("models/NCBI_NER_2_epoch") ## load existing model
    
ner_pipeline = Pipeline(stages=[
      clinical_embeddings,
      ner_graph_builder,
      nerTagger
 ])

In [21]:
# remove the existing logs

! rm -r ./ner_logs

In [22]:
%%time
ner_model_retrained = ner_pipeline.fit(test_data_1)

TF Graph Builder configuration:
Model name: ner_dl
Graph folder: medical_ner_graphs
Graph file name: auto
Build params: {'ntags': 3, 'embeddings_dim': 200, 'nchars': 79, 'is_medical': True, 'lstm_size': 24}
ner_dl graph exported to medical_ner_graphs/blstm_3_200_24_79.pb
CPU times: user 11.5 s, sys: 320 ms, total: 11.8 s
Wall time: 31.1 s


In [23]:
!ls ner_logs/MedicalNerApproach*

ner_logs/MedicalNerApproach_bc2e5164bf39.log


In [24]:
!cat ./ner_logs/MedicalNerApproach*

Name of the selected graph: pretrained graph
Training started - total epochs: 2 - lr: 0.003 - batch size: 8 - labels: 3 - chars: 84 - training examples: 434


Epoch 1/2 started, lr: 0.003, dataset size: 434


Epoch 1/2 - 5.55s - loss: 79.691826 - avg training loss: 1.4230683 - batches: 56
Quality on test dataset: 
time to finish evaluation: 1.69s
Total test loss: 59.8559	Avg test loss: 0.9069
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 463	 43	 120	 0.91501975	 0.7941681	 0.8503214
B-Disease	 447	 81	 71	 0.84659094	 0.86293435	 0.8546845
tp: 910 fp: 124 fn: 191 labels: 2
Macro-average	 prec: 0.8808054, rec: 0.82855123, f1: 0.85387963
Micro-average	 prec: 0.88007736, rec: 0.82652134, f1: 0.852459


Epoch 2/2 started, lr: 0.0029850747, dataset size: 434


Epoch 2/2 - 3.55s - loss: 62.109985 - avg training loss: 1.1091069 - batches: 56
Quality on test dataset: 
time to finish evaluation: 1.20s
Total test loss: 61.8775	Avg test loss: 0.9375
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 499	

In [25]:
from sparknlp_jsl.eval import NerDLMetrics

pred_df = ner_model_retrained.stages[2].transform(clinical_embeddings.transform(test_data_2))

evaler = NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label",case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
    .withColumn("recall", F.round(eval_result["recall"],4))\
    .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+-----+----+-----+---------+------+------+
| entity|   tp|   fp|  fn|total|precision|recall|    f1|
+-------+-----+-----+----+-----+---------+------+------+
|Disease|443.0|102.0|72.0|515.0|   0.8128|0.8602|0.8358|
+-------+-----+-----+----+-----+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.8358490566037736|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.8358490566037736|
+------------------+

None


### Train a Model Choosing Best Model

`useBestModel`:  This param preserves and restores the model that has achieved the best performance at the end of the training. The priority is metrics from testDataset (micro F1), metrics from validationSplit (micro F1), and if none is set it will keep track of loss during the training. <br/>

`earlyStopping`: You can stop training at the point when the perforfmance on test/validation dataset starts to degrage. Two params are used in order to use this feature: <br/> 

- `earlyStoppingCriterion`: This is used set the minimal improvement of the test metric to terminate training. The metric monitored is the same as the metrics used in `useBestModel` (micro F1 when using test/validation set, loss otherwise). Default is 0 which means no early stopping is applied. 

- `earlyStoppingPatience`:  The number of epoch without improvement which will be tolerated. Default is 0, which means that early stopping will occur at the first time when performance in the current epoch is no better than in the previous epoch.

In [26]:
# remove the existing logs

! rm -r ./ner_logs

In [27]:
nerTagger = medical.NerApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(20)\
      .setLr(0.003)\
      .setBatchSize(8)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setTestDataset('./test_2.parquet')\
      .setGraphFolder(graph_folder_path)\
      .setOutputLogsPath('./ner_logs')\
      .setValidationSplit(0.2)\
      .setUseBestModel(True)\
      .setEarlyStoppingCriterion(0.04)\
      .setEarlyStoppingPatience(3)


ner_pipeline = Pipeline(stages=[
      clinical_embeddings,
      ner_graph_builder,
      nerTagger
 ])

In [28]:
%%time
ner_model = ner_pipeline.fit(training_data)

TF Graph Builder configuration:
Model name: ner_dl
Graph folder: medical_ner_graphs
Graph file name: auto
Build params: {'ntags': 3, 'embeddings_dim': 200, 'nchars': 85, 'is_medical': True, 'lstm_size': 24}
ner_dl graph exported to medical_ner_graphs/blstm_3_200_24_85.pb
CPU times: user 11.6 s, sys: 369 ms, total: 12 s
Wall time: 3min 27s


In [29]:
!ls ner_logs/MedicalNerApproach*

ner_logs/MedicalNerApproach_610829d8d406.log


In [30]:
# Training Logs
! cat ner_logs/MedicalNerApproach*

Name of the selected graph: /content/medical_ner_graphs/blstm_3_200_24_85.pb
Training started - total epochs: 20 - lr: 0.003 - batch size: 8 - labels: 3 - chars: 84 - training examples: 5060


Epoch 1/20 started, lr: 0.003, dataset size: 5060


Epoch 1/20 - 44.50s - loss: 1799.438 - avg training loss: 2.8337605 - batches: 635
Quality on validation dataset (20.0%), validation examples = 1012
time to finish evaluation: 4.44s
Total validation loss: 232.1383	Avg validation loss: 1.4242
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 1043	 138	 445	 0.88314986	 0.70094085	 0.78156614
B-Disease	 924	 144	 266	 0.8651685	 0.7764706	 0.8184234
tp: 1967 fp: 282 fn: 711 labels: 2
Macro-average	 prec: 0.8741592, rec: 0.73870575, f1: 0.8007446
Micro-average	 prec: 0.87461096, rec: 0.7345034, f1: 0.7984575
Quality on test dataset: 
time to finish evaluation: 1.63s
Total test loss: 106.3521	Avg test loss: 1.6114
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 437	 58	 146	 0.8828283	 0.7495712	 0.8107607
B-

As you see above, training was stopped before 20th epoch since there were not improvement.

**Evaluate**

In [31]:
from sparknlp_jsl.eval import NerDLMetrics

pred_df = ner_model.stages[2].transform(clinical_embeddings.transform(test_data_1))

evaler = NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
    .withColumn("recall", F.round(eval_result["recall"],4))\
    .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+----+-----+---------+------+------+
| entity|   tp|  fp|  fn|total|precision|recall|    f1|
+-------+-----+----+----+-----+---------+------+------+
|Disease|362.0|84.0|78.0|440.0|   0.8117|0.8227|0.8172|
+-------+-----+----+----+-----+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.8171557562076749|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.8171557562076749|
+------------------+

None


In [32]:
pred_df = ner_model.stages[2].transform(clinical_embeddings.transform(test_data_2))

evaler = NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
    .withColumn("recall", F.round(eval_result["recall"],4))\
    .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+----+-----+---------+------+------+
| entity|   tp|  fp|  fn|total|precision|recall|    f1|
+-------+-----+----+----+-----+---------+------+------+
|Disease|447.0|82.0|68.0|515.0|    0.845| 0.868|0.8563|
+-------+-----+----+----+-----+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.8563218390804598|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.8563218390804598|
+------------------+

None


## Now let's take a model trained on a different dataset and train on this dataset

In [33]:
jsl_ner = medical.NerModel.pretrained('ner_jsl','en','clinical/models')

jsl_ner.getClasses()

ner_jsl download started this may take some time.
[OK!]


['O',
 'B-Injury_or_Poisoning',
 'B-Direction',
 'B-Test',
 'I-Route',
 'B-Admission_Discharge',
 'B-Death_Entity',
 'I-Oxygen_Therapy',
 'I-Drug_BrandName',
 'B-Relationship_Status',
 'B-Duration',
 'I-Alcohol',
 'I-Triglycerides',
 'I-Date',
 'B-Respiration',
 'B-Hyperlipidemia',
 'I-Test',
 'B-Birth_Entity',
 'I-VS_Finding',
 'B-Age',
 'I-Social_History_Header',
 'B-Labour_Delivery',
 'I-Medical_Device',
 'B-Family_History_Header',
 'B-BMI',
 'I-Fetus_NewBorn',
 'I-BMI',
 'B-Temperature',
 'I-Section_Header',
 'I-Communicable_Disease',
 'I-ImagingFindings',
 'I-Psychological_Condition',
 'I-Obesity',
 'I-Sexually_Active_or_Sexual_Orientation',
 'I-Modifier',
 'B-Alcohol',
 'I-Temperature',
 'I-Vaccine',
 'I-Symptom',
 'B-Kidney_Disease',
 'I-Pulse',
 'B-Oncological',
 'I-EKG_Findings',
 'B-Medical_History_Header',
 'I-Relationship_Status',
 'I-Blood_Pressure',
 'B-Cerebrovascular_Disease',
 'I-Diabetes',
 'B-Oxygen_Therapy',
 'B-O2_Saturation',
 'B-Psychological_Condition',
 'B-Hear

### Now train a model using this model as base

In [34]:
nerTagger = medical.NerApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(2)\
      .setLr(0.003)\
      .setBatchSize(8)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setTestDataset('/content/test_2.parquet')\
      .setOutputLogsPath('ner_logs')\
      .setGraphFolder(graph_folder_path)\
      .setPretrainedModelPath("/root/cache_pretrained/ner_jsl_en_3.1.0_3.0_1624284761441")\
      .setOverrideExistingTags(True) # since the tags do not align, set this flag to true
    
# do hyperparameter by tuning the params above (max epoch, LR, dropout etc.) to get better results
ner_pipeline = Pipeline(stages=[
      clinical_embeddings,
      ner_graph_builder,
      nerTagger
 ])

In [35]:
# remove the existing logs

! rm -r ./ner_logs

In [36]:

%%time
ner_jsl_retrained = ner_pipeline.fit(training_data)


TF Graph Builder configuration:
Model name: ner_dl
Graph folder: medical_ner_graphs
Graph file name: auto
Build params: {'ntags': 3, 'embeddings_dim': 200, 'nchars': 85, 'is_medical': True, 'lstm_size': 24}
ner_dl graph exported to medical_ner_graphs/blstm_3_200_24_85.pb
CPU times: user 13 s, sys: 437 ms, total: 13.4 s
Wall time: 7min 28s


In [37]:
!cat ./ner_logs/MedicalNerApproach*

Name of the selected graph: pretrained graph
Training started - total epochs: 2 - lr: 0.003 - batch size: 8 - labels: 3 - chars: 106 - training examples: 6347


Epoch 1/2 started, lr: 0.003, dataset size: 6347


Epoch 1/2 - 193.12s - loss: 1932.797 - avg training loss: 2.4311912 - batches: 795
Quality on test dataset: 
time to finish evaluation: 14.61s
Total test loss: 75.0408	Avg test loss: 1.1370
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 480	 64	 103	 0.88235295	 0.8233276	 0.8518189
B-Disease	 426	 77	 92	 0.84691846	 0.82239383	 0.83447605
tp: 906 fp: 141 fn: 195 labels: 2
Macro-average	 prec: 0.8646357, rec: 0.8228607, f1: 0.84323114
Micro-average	 prec: 0.8653295, rec: 0.82288826, f1: 0.8435754


Epoch 2/2 started, lr: 0.0029850747, dataset size: 6347


Epoch 2/2 - 195.17s - loss: 913.9067 - avg training loss: 1.1495682 - batches: 795
Quality on test dataset: 
time to finish evaluation: 14.95s
Total test loss: 56.4276	Avg test loss: 0.8550
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Dis

In [38]:
from sparknlp_jsl.eval import NerDLMetrics

pred_df = ner_jsl_retrained.stages[2].transform(clinical_embeddings.transform(test_data_2))

evaler = NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
    .withColumn("recall", F.round(eval_result["recall"],4))\
    .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+----+-----+---------+------+------+
| entity|   tp|  fp|  fn|total|precision|recall|    f1|
+-------+-----+----+----+-----+---------+------+------+
|Disease|425.0|87.0|90.0|515.0|   0.8301|0.8252|0.8277|
+-------+-----+----+----+-----+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.8276533592989289|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.8276533592989289|
+------------------+

None
